In [ ]:
torchrun --standalone --nproc_per_node=1 train_sft.py \
    --pretrain "./checkpoint/llama_7B" \
    --model 'llama' \
    --strategy colossalai_zero2 \
    --log_interval 10 \
    --save_path "./checkpoint_mine/step1/epoch1" \
    --dataset 'PersonaChat' \
    --batch_size 2 \
    --accumulation_steps 8 \
    --lr 2e-5 \
    --max_epochs 1 \
    --max_datasets_size 1024 \
    --batch_size 2 \

In [ ]:
torchrun --standalone --nproc_per_node=1 train_reward_model.py \
    --strategy colossalai_zero2 \
    --model 'bloom' \
    --pretrain 'bigscience/bloom-560m' \
    --dataset 'PersonaChat' \
    --save_path './checkpoint_mine/step2/epoch1/rmstatic.pt' \
    --max_epochs 1 \
    --batch_size 8 \
    --loss_fn 'log_exp' \
    --max_datasets_size 256

In [ ]:
torchrun --standalone --nproc_per_node=1 train_prompts.py \
    --prompt_dataset 'datasets/seed_prompts_en.jsonl' \
    --pretrain_dataset 'colossalai_gemini' \
    --strategy colossalai_zero2 \
    --model 'llama' \
    --pretrain './checkpoint_mine/step1/epoch1' \
    --rm_model 'llama' \
    --rm_path './checkpoint_mine/step2/epoch1/rmstatic.pt' \
    --save_path './checkpoint_mine/step3/epoch1' \
    --max_epochs 1 \
    --train_batch_size 1 \
    --ptx_batch_size 1 \
    --lora_rank 16 \
    --max_input_len 96 \
    --max_seq_len 128

In [ ]:
https://github.com/hpcaitech/ColossalAI/issues/3192
Traceback (most recent call last):
  File "/home/jovyan/work/Chat/train_prompts.py", line 266, in <module>
    main(args)
  File "/home/jovyan/work/Chat/train_prompts.py", line 79, in main
    reward_model.load_state_dict(state_dict)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1671, in load_state_dict
    raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(
RuntimeError: Error(s) in loading state_dict for LlamaRM:
        Unexpected key(s) in state_dict: "model.layers.0.self_attn.q_proj.lora_A",

In [ ]:
I have the same problems.
I add the `strict=False` to `reward_model.load_state_dict(state_dict)` and `critic.load_state_dict(state_dict)` can solve the problems

In [ ]:
import torch
import torch.utils.data as data
from torch.utils.data import DataLoader

dataset = data.TensorDataset(torch.randn(10, 3), torch.randn(10, 1))
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

for i, (x_batch, y_batch) in enumerate(dataloader):
    print(f"Batch {i}:\n{x_batch}\n{y_batch}")

In [ ]:
class PromptDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self,
                 data_path: str,
                 tokenizer: transformers.PreTrainedTokenizer,
                 max_datasets_size: int = None,
                 max_length: int = 96):
        super(PromptDataset, self).__init__()
        self.keyed_prompt = defaultdict(list)
        logger.info("Loading data...")
        list_data_dict = jload(data_path)
        logger.info(f"Loaded {len(list_data_dict)} examples.")

        if max_datasets_size is not None:
            logger.info(f"Limiting dataset to {max_datasets_size} examples.")
            list_data_dict = list_data_dict[:max_datasets_size]

        for data_dict in list_data_dict:
            token = tokenizer(data_dict["instruction"],
                              return_tensors='pt',
                              max_length=max_length,
                              padding='max_length',
                              truncation=True)
            for k, tensor in token.items():
                self.keyed_prompt[k].extend(tensor.to(torch.cuda.current_device()).unbind())

    def __len__(self):
        return len(self.keyed_prompt["input_ids"])

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return {k: v[i] for k, v in self.keyed_prompt.items()}

In [ ]:
from dataprocess import create_data
inputs = []
outputs = []
persona, query, response, _ = create_data("./datasets/convai/train_self_original.txt")
print(query[0][:10])
print(response[0][:10])
for i in range(len(persona)):
    dialogue = []
    dialogue.append(query[i][0])
    inputs.append("\n".join(persona[i]+[query[i][0]]))
    outputs.append(response[i][0])
    
    for j in range(len(query[i])-1):
        dialogue.append(response[i][j])
        dialogue.append(query[i][j+1])
        inputs.append("\n".join(persona[i] + dialogue))
        outputs.append(response[i][j+1])

In [ ]:
import itertools
for i in itertools.count(10,2):
    print(i)
    if i>20: 
        break

In [ ]:
from dataprocess import create_data
def PersonaPromptDataset(data_file, max_datasets_size):
    dataset_chosen = []
    dataset_rejected = []

    persona, query, response, cand = create_data(data_file, max_datasets_size)
    for i in range(len(persona)):
        for j in range(len(persona[i])):
            dataset_chosen.append("\n".join(persona[i][:j] + 
                                            list(itertools.chain.from_iterable(zip(query[i][:j],response[i][:j])))) + "\n")
            dataset_rejected.append("\n".join(persona[i][:j] + 
                                            list(itertools.chain.from_iterable(zip(query[i][:j],cand[i][:j][0])))) + "\n")

    return dataset_chosen, dataset_rejected

In [1]:
from dataprocess import HhRlhfDataset, PersonaPromptDataLoader, create_data, PersonaPromptDataset
dataset_chosen, dataset_rejected = PersonaPromptDataset("./datasets/convai/valid_self_original.txt", 999999)

1000 1000 1000 1000
./datasets/convai/valid_self_original.txt has 1000 dialog and 7801 query


In [2]:
for i in range(10):
    print(inputs[i])
    print("-"*25)
    print(outputs[i])
    print("-"*50)

NameError: name 'inputs' is not defined

In [ ]:
import torch
print(torch.version.cuda)

In [ ]:
import itertools
def create_data(data_file):
    with open(data_file, "r", encoding="utf8") as f:
        persona =[]
        query = []
        response = []
        cand = []
        is_persona = False
        tmp_persona = []
        tmp_query = []
        tmp_response = []
        tmp_cand = []
        first = True
        cnt = 0
        sum_u = 0
        for line in f:
            cnt += 1
            line = line.strip()
            if "your persona: " in line:
                if not is_persona and not first:
                    query.append(tmp_query)
                    response.append(tmp_response)
                    cand.append(tmp_cand)
                    sum_u += len(tmp_query)
                    tmp_query = []
                    tmp_response = []
                    tmp_cand = []
                first = False
                is_persona = True
                line = line.split(": ", maxsplit=1)[1]
                tmp_persona.append(line)
            else:
                if is_persona:
                    persona.append(tmp_persona)
                    is_persona = False
                    tmp_persona = []
                line = line[line.find(" ")+1:]
                tmp_query.append(line.split("\t")[0])
                tmp_response.append(line.split("\t")[1])
                tmp_cand.append(line.split("\t")[3].split("|"))
        query.append(tmp_query)
        response.append(tmp_response)
        cand.append(tmp_cand)
        sum_u += len(tmp_query)
        assert len(query) == len(response) == len(persona) == len(cand)

    print("{} has {} dialog and {} query".format(data_file, len(query), sum_u))

    return persona, query, response, cand

In [ ]:
train_dataset_chosen = []
train_dataset_rejected = []
eval_dataset_chosen = []
eval_dataset_rejected = []

persona, query, response, cand = create_data(f"./datasets/convai/valid_self_original.txt")
print(persona[:2])
print()
print(query[:2])
print()
print(cand[:2])

for i in range(len(persona)):
    for j in range(len(cand[i])):
        eval_dataset_chosen.append("\n".join(persona[i] + list(itertools.chain.from_iterable(zip(query[i],response[i])))) + "\n")
        eval_dataset_rejected.append("\n".join(persona[i] + list(itertools.chain.from_iterable(zip(query[i],cand[i][j])))) + "\n")
train_dataset_chosen = []
train_dataset_rejected = []
eval_dataset_chosen = []
eval_dataset_rejected = []

persona, query, response, cand = create_data(f"./datasets/convai/train_self_original.txt")
for i in range(len(persona)):
    for j in range(len(cand[i])):
        train_dataset_chosen.append("\n".join(persona[i] + list(itertools.chain.from_iterable(zip(query[i],response[i])))) + "\n")
        train_dataset_rejected.append("\n".join(persona[i] + list(itertools.chain.from_iterable(zip(query[i],cand[i][j])))) + "\n")

In [ ]:
print(eval_dataset_chosen[2])

In [ ]:
print(eval_dataset_rejected[:3])

In [ ]:
import random

my_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
percentage = 0.2

random_list = random.sample(my_list, int(len(my_list) * percentage))
print(random_list)